<a href="https://colab.research.google.com/github/Jashwanth63/Custom-Model-for-classification-of-Image-Data-without-using-Convolutional-Neural-Networks/blob/main/DLL_Assingment_TYPE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [234]:
import pickle
import torch
import torch.nn as nn
from torchvision.datasets import FashionMNIST
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset
import torch.nn.functional as F
import torch.optim as optim
import tqdm as tqdm

In [235]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

In [236]:
device

device(type='cuda', index=0)

In [237]:
def write_vectorised(data, data_usecase):
  if data_usecase == "TRAIN":
    with open("vectorised_train.txt", "wb") as file_pointer:
      pickle.dump(data, file_pointer)
  elif data_usecase == "TEST":
    with open("vectorised_test.txt","wb") as file_pointer:
      pickle.dump(data, file_pointer)
      
def read_vectorised(data_usecase):
  if data_usecase == "TRAIN":
    with open("vectorised_train.txt", "rb") as file_pointer:
      return pickle.load(file_pointer)
  elif data_usecase == "TEST":
    with open("vectorised_test.txt","rb") as file_pointer:
      return pickle.load(file_pointer)


In [238]:
def download_data(b_size=64):  
    """Download the Fashion-MNIST dataset and then load it into memory."""
    mnist_train = torchvision.datasets.FashionMNIST(".", train = True, transform = transforms.ToTensor(), target_transform= transforms.Lambda(lambda x:torch.zeros(
            10, dtype = torch.float64).scatter_(dim = 0, index= torch.tensor(x), value = 1)) ,download = True)
    mnist_train = torch.utils.data.DataLoader(mnist_train, batch_size = b_size, shuffle = True, num_workers = 3)

    mnist_test = torchvision.datasets.FashionMNIST(".", train = False, transform = transforms.ToTensor(), target_transform = transforms.Lambda(
        lambda x:torch.zeros(10, dtype = torch.float64).scatter_(dim=0, index = torch.tensor(x), value = 1)), download = True)
    mnist_test = torch.utils.data.DataLoader(mnist_test, batch_size=b_size, shuffle = True, num_workers = 3)
    return mnist_train, mnist_test  

In [239]:
training,  testing = download_data(b_size=64)

In [7]:
def patch_tensor_maker(image, patch_size, im_size):
  elements = []
  for l in range(0, im_size-patch_size+1, patch_size):
    for k in range(0, im_size-patch_size+1, patch_size):
      patch = []
      for i in range(patch_size):
        temp = []
        for j in range(patch_size):
          temp.append(image[l+i][k+j])
        patch.append(temp)
      elements.append(patch)
  return elements


def patch_maker(data, patch_size = 4):
  data_list_main = list()
  for i, batch in enumerate(data):
    batch_vectorised = []
    for image in batch[0]:
      im_size = image.size()[1]
      batch_vectorised.append(patch_tensor_maker(image.view(im_size, im_size), patch_size, im_size))
    main_data = torch.Tensor(batch_vectorised)
    main_labels = batch[1]
    data_list_main.append([main_data, main_labels])
    #data_list_main= TensorDataset([main_data, main_labels])
  #return torch.utils.data.DataLoader(iter(data_list_main))
  return data_list_main

In [9]:
train_data = patch_maker(training)
test_data = patch_maker(testing)

In [10]:
#Vectorize (4,4) => (16)
for i in range(len(train_data)):
    train_data[i][0] = train_data[i][0].view(-1, 49, 16) 

for i in range(len(test_data)):
    test_data[i][0] = test_data[i][0].view(-1, 49, 16) 

In [13]:
write_vectorised(train_data, data_usecase = "TRAIN")
write_vectorised(test_data, data_usecase = "TEST")


In [240]:
train_data = read_vectorised(data_usecase = "TRAIN") 
test_data = read_vectorised(data_usecase = "TEST") 

In [40]:
[1,2,3]*2

[1, 2, 3, 1, 2, 3]

In [65]:
3%2

1

In [67]:
1%2

1

In [241]:
class net(nn.Module):
  def __init__(self, block_size):
    super(net, self).__init__()

    #STEM
    self.linear = nn.Linear(16, 512)

    #BACKBONE
    self.blocks = block_size


    #self.mlp_1 = nn.ModuleList([nn.Linear(49, 49), nn.Linear(49, 49)])
    #self.mlp_2 = nn.ModuleList([nn.Linear(256, 256), nn.Linear(256,256)])

    #self.block = nn.ModuleList([self.mlp_1, self.mlp_2]*2)

    #self.hidden1 = nn.ModuleList([nn.Linear(49,49) for i in range(self.blocks)])
    #self.hidden2 = nn.ModuleList([nn.Linear(49,49) for i in range(self.blocks)])
    #self.hidden3 = nn.ModuleList([nn.Linear(256,256) for i in range(self.blocks)])
    #self.hidden4 = nn.ModuleList([nn.Linear(256, 256) for i in range(self.blocks)])

    self.block1 = nn.ModuleList([nn.Linear(49,49), nn.Linear(49,49), nn.Linear(512,512),
                                 nn.Linear(512,512)])
    self.block2 = nn.ModuleList([nn.Linear(49,49), nn.Linear(49,49), nn.Linear(512,512),
                                 nn.Linear(512,512)])
    self.block3 = nn.ModuleList([nn.Linear(49,49), nn.Linear(49,49), nn.Linear(512, 512),
                                 nn.Linear(512,512)])


    #CLASSIFIER
    self.output = nn.Linear(512, 10)
    self.probs = nn.Softmax(dim=1)


  def forward(self, x):
    #STEM
    x = self.linear(x)

    #BACKBONE
    x = self.block_caller(x, self.block1)
    x = self.block_caller(x, self.block2)
    #x = self.block_caller(x, self.block3)

    #CLASSIFIER
    x = torch.sum(x, dim=1) / x[1].size()[0]
    x = self.output(x)
    x = self.probs(x)
    return x


  def block_caller(self, x, block):
    #BACKBONE - EACH BLOCK HERE HAS 2 MLPS
    for i, layers in enumerate(block):
      #h1 = self.hidden1[i]
      #h2 = self.hidden2[i]
      if i==0:
        x = x.transpose(1,2)  #X=> X^T
        x = F.tanh(block[i](x)) #g(X^T,W1) , g() is relu()
      elif i==1:
        x = block[i](x) #g() X W2
      
      #h3 = self.hidden3[i]
      #h4 = self.hidden4[i]
      elif i==2:
        x = x.transpose(1,2) #O1 => O1^T
        x = F.tanh(block[i](x))  #g(O1, W3), g() is relu()
      elif i==3:
        x = block[i](x) #g() X W4
    return x

  

In [242]:
NET = net(block_size=1).to(device)

In [243]:
optimizer = optim.SGD(NET.parameters(), lr = 0.1)
loss_function = nn.CrossEntropyLoss()

In [244]:
def accuracy_calculator(hyp, y):
  a = torch.argmax(hyp, dim=1)
  b = torch.argmax(y, dim=1)
  return torch.sum(a==b)

In [245]:
def train(out):
  EPOCHS = 100
  accuracy = 0
  for i in range(EPOCHS):
      accuracy = 0
      for i in tqdm.tqdm(range(len(train_data))): 
          X, y = train_data[i]
          X = X.to(device)
          y = y.to(device)
          out.zero_grad()
          hypothesis = out(X)
          hypothesis.to(device)
          accuracy += accuracy_calculator(hypothesis, y)
          loss = loss_function(hypothesis, y)
          loss.backward()
          optimizer.step()

      print("LOSS is: ",loss)
      print("Accuracy is: ",accuracy/60000)
  

def test(out):
  accuracy = 0
  with torch.no_grad():
    for i in tqdm.tqdm(range(len(test_data))):
          X, y = test_data[i]
          X = X.to(device)
          y = y.to(device)
          hypothesis = out(X)
          hypothesis.to(device)
          accuracy += accuracy_calculator(hypothesis, y)
    print(accuracy/10000)

In [246]:
train(NET)

  0%|          | 0/938 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 938/938 [00:06<00:00, 148.70it/s]


LOSS is:  tensor(2.3032, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>)
Accuracy is:  tensor(0.0994, device='cuda:0')


100%|██████████| 938/938 [00:06<00:00, 149.16it/s]


LOSS is:  tensor(2.3030, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>)
Accuracy is:  tensor(0.0996, device='cuda:0')


100%|██████████| 938/938 [00:06<00:00, 148.96it/s]


...

LOSS is:  tensor(1.6486, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>)
Accuracy is:  tensor(0.7702, device='cuda:0')


100%|██████████| 938/938 [00:06<00:00, 148.90it/s]


LOSS is:  tensor(1.6487, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>)
Accuracy is:  tensor(0.7694, device='cuda:0')


100%|██████████| 938/938 [00:06<00:00, 148.94it/s]

LOSS is:  tensor(1.6487, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>)
Accuracy is:  tensor(0.7704, device='cuda:0')


In [247]:
test(NET)

  0%|          | 0/157 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 157/157 [00:00<00:00, 418.54it/s]

tensor(0.7532, device='cuda:0')


In [ ]:
#NOTE:
#1. EPOCHS =100, lr =0.1, BLOCKS =2, accuracy: 73 (Train), 72(Test), Activation: Relu
#2. EPOCHS =100, lr =0.1, BLOCKS =1, accuracy: 74.93 (Train), 73.18(Test), Activation: Relu
#2. EPOCHS =100, lr =0.1, BLOCKS =2, accuracy: 84.82 (Train), 82.34(Test), Activation: TanH
#2. EPOCHS =100, lr =0.13, BLOCKS =3, accuracy: 81.56 (Train), 80.02(Test), Activation: TanH